In [1]:
import ee
import geemap

# Authentifier et initialiser Google Earth Engine
ee.Authenticate()
ee.Initialize(project="project-2026-476714")


Enter verification code: 4/1Ab32j902UTHch7ojdUXS3da-HlcHXWx3vDmAgV02xVkBKtPhWxvFOsNOA8w

Successfully saved authorization token.


In [1]:
import ee
import geemap

In [2]:
import ee
ee.Initialize(project="project-2026-476714")

print("Earth Engine est connecté :", ee.Number(1).getInfo())


Earth Engine est connecté : 1


In [3]:
# ===== 2.1. Préfixe des assets =====
ASSET = "projects/project-2026-476714/assets/"

def fc(name):
    """Raccourci pour créer une FeatureCollection depuis ton projet."""
    return ee.FeatureCollection(ASSET + name)

# ===== 2.2. GADM : limites administratives du Tchad =====
tcd0 = fc("gadm41_TCD_0")  # pays
tcd1 = fc("gadm41_TCD_1")  # régions
tcd2 = fc("gadm41_TCD_2")  # départements
tcd3 = fc("gadm41_TCD_3")  # sous-préfectures

# ===== 2.3. OSM : couches principales =====
buildings = fc("gis_osm_buildings_a_free_1")
landuse   = fc("gis_osm_landuse_a_free_1")
natural_a = fc("gis_osm_natural_a_free_1")
natural_l = fc("gis_osm_natural_free_1")
places_a  = fc("gis_osm_places_a_free_1")
places_p  = fc("gis_osm_places_free_1")
pois_a    = fc("gis_osm_pois_a_free_1")
pois_p    = fc("gis_osm_pois_free_1")
pofw_a    = fc("gis_osm_pofw_a_free_1")
pofw_p    = fc("gis_osm_pofw_free_1")
railways  = fc("gis_osm_railways_free_1")
roads     = fc("gis_osm_roads_free_1")
traffic_a = fc("gis_osm_traffic_a_free_1")
traffic_l = fc("gis_osm_traffic_free_1")
transport_a = fc("gis_osm_transport_a_free_1")
transport_l = fc("gis_osm_transport_free_1")
waterways_l = fc("gis_osm_waterways_free_1")
waterways_a = fc("gis_osm_waterways_a_free_1")

print("Collections chargées.")
pop100 = ee.Image("users/bamba/tcd_pop_2025_100m")
pop1km = ee.Image("users/bamba/tcd_pop_2025_1km")


Collections chargées.


In [4]:
pop100 = ee.Image("projects/project-2026-476714/assets/tcd_pop_2025_CN_100m_R2025A_v1")
pop1km = ee.Image("projects/project-2026-476714/assets/tcd_pop_2025_CN_1km_R2025A_UA_v1")

In [8]:
import ee
import geemap

ee.Initialize()

# Raster 100 m limité au Tchad
pop100_tchad = pop100.clip(tcd0)

# Style des régions
regions_style = {
    'color': 'blue',
    'width': 2,
    'fillColor': '00000000'   # transparent
}

# Carte
m = geemap.Map()
m.add_basemap('OpenStreetMap')
m.center_object(tcd0, 5)

# 1) Raster 100 m en dessous
vis_pop = {
    'min': 0,
    'max': 1000,
    'palette': ['blue','green','yellow','orange','red']
}
m.addLayer(pop100_tchad, vis_pop, "Population 100 m")

# 2) Limites régionales par-dessus
m.addLayer(tcd1.style(**regions_style), {}, "Régions (GADM 1)")

output_html = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\map_population100.html"
m.to_html(output_html)
m


Map(center=[15.306204623659646, 18.656507734823464], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
import ee
import geemap

ee.Initialize()

# Raster 1 km limité au Tchad
pop1km_tchad = pop1km.clip(tcd0)

# Style des régions
regions_style = {
    'color': 'blue',
    'width': 2,
    'fillColor': '00000000'   # transparent
}

# Carte
m = geemap.Map()
m.add_basemap('OpenStreetMap')
m.center_object(tcd0, 5)

# 1) Raster 1 km en dessous
vis_pop_1km = {
    'min': 0,
    'max': 1000,
    'palette': ['blue','green','yellow','orange','red']
}
m.addLayer(pop1km_tchad, vis_pop_1km, "Population 1 km")

# 2) Limites régionales par-dessus
m.addLayer(tcd1.style(**regions_style), {}, "Régions (GADM 1)")
output_html = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\map_population1000.html"
m.to_html(output_html)
m


Map(center=[15.306204623659646, 18.656507734823464], controls=(WidgetControl(options=['position', 'transparent…

In [11]:
# ===== 4.1. Préparer les filtres POI =====

# Hypothèse : champ 'fclass' contient la catégorie OSM.
# Adapte les valeurs si besoin après avoir inspecté quelques features.

# Santé : pharmacies, hôpitaux, cliniques...
health_classes = ['pharmacy', 'hospital', 'clinic', 'doctors']

health_pois = pois_p.filter(ee.Filter.inList('fclass', health_classes))

# Éducation : écoles, collèges, universités...
school_classes = ['school', 'college', 'university']

school_pois = pois_p.filter(ee.Filter.inList('fclass', school_classes))

# Lieux de culte : directement dans pofw_p (points)
worship_pois = pofw_p  # déjà spécifique


# ===== 4.2. Fonction utilitaire pour compter les points par région =====

def add_point_count(regions_fc, points_fc, field_name):
    """
    Ajoute à chaque feature de regions_fc une propriété field_name
    contenant le nombre de points_fc dans la géométrie de la région.
    """
    def _count_points(region):
        pts_in_region = points_fc.filterBounds(region.geometry())
        return region.set(field_name, pts_in_region.size())
    return regions_fc.map(_count_points)


# ===== 4.3. Appliquer aux régions =====

regions_stats = tcd1

regions_stats = add_point_count(regions_stats, health_pois, 'nb_sante')
regions_stats = add_point_count(regions_stats, school_pois, 'nb_ecoles')
regions_stats = add_point_count(regions_stats, worship_pois, 'nb_culte')

# Vérifier une région en particulier (premier élément)
first_region = regions_stats.first()
print(first_region.toDictionary().getInfo())


{'CC_1': 'NA', 'COUNTRY': 'Chad', 'ENGTYPE_1': 'Region', 'GID_0': 'TCD', 'GID_1': 'TCD.23_1', 'HASC_1': 'TD.BI', 'ISO_1': 'NA', 'NAME_1': 'Wadi Fira', 'NL_NAME_1': 'NA', 'TYPE_1': 'Région', 'VARNAME_1': 'Biltine', 'nb_culte': 2, 'nb_ecoles': 3, 'nb_sante': 14}


In [13]:
import os,pandas as pd

# 1) Récupérer la FeatureCollection côté client (ATTENTION : à faire seulement
#    pour de petites collections, comme ici)
regions_info = regions_stats.getInfo()

# 2) Extraire la liste des "features"
features = regions_info['features']

# 3) Construire une liste de dictionnaires avec les propriétés uniquement
rows = [f['properties'] for f in features]

# 4) Créer le DataFrame pandas
df = pd.DataFrame(rows)

output_path = r"C:\Users\admin\Pictures\ISEP3\TP2\outputs"
os.makedirs(output_path, exist_ok=True)

# Chemins des fichiers
csv_path  = os.path.join(output_path, "services_sociaux_par_region.csv")
xlsx_path = os.path.join(output_path, "services_sociaux_par_region.xlsx")

# Sauvegarde
df.to_csv(csv_path, index=False)
df.to_excel(xlsx_path, index=False)

print("Fichiers enregistrés :")
print(csv_path)
print(xlsx_path)
# 5) Afficher seulement quelques colonnes intéressantes
df[['NAME_1', 'nb_sante', 'nb_ecoles', 'nb_culte']]


Fichiers enregistrés :
C:\Users\admin\Pictures\ISEP3\TP2\outputs\services_sociaux_par_region.csv
C:\Users\admin\Pictures\ISEP3\TP2\outputs\services_sociaux_par_region.xlsx


,NAME_1,nb_sante,nb_ecoles,nb_culte
0,Wadi Fira,14,3,2
1,Borkou,4,9,3
2,Guéra,4,14,4
3,Logone Occidental,5,7,7
4,Logone Oriental,14,14,13
5,Ville de N'Djamena,105,115,33
6,Barh el Ghazel,11,3,6
7,Batha,7,0,1
8,Ennedi Est,0,0,0
9,Ennedi Ouest,0,1,0


In [14]:

# Ton raccourci fc(...) et tes collections sont déjà définis plus haut

# Filtrer les POI pour ne garder que les services de santé
sante = pois_p.filter(
    ee.Filter.inList(
        'fclass',
        ['hospital', 'clinic', 'doctors', 'pharmacy', 'dentist']
    )
)

print("Nombre de points de santé :", sante.size().getInfo())



Nombre de points de santé : 340


In [15]:
regions_stats = tcd1.map(
    lambda region: region.set(
        "nb_sante",
        sante.filterBounds(region.geometry()).size()
    )
)


In [19]:
import ee, geemap
ee.Initialize()

# === 1) CHOROPLÈTHE nb_sante ===
nb_sante_img = regions_stats.reduceToImage(
    properties=['nb_sante'],
    reducer=ee.Reducer.first()
)

max_nb = regions_stats.aggregate_max('nb_sante').getInfo()
print("nb_sante max :", max_nb)

# Palette bleue pour nb_sante
vis_nb = {
    'min': 0,
    'max': max_nb,
    'palette': ['#f7fbff','#c6dbef','#6baed6','#2171b5','#08306b']
}

# Style limites
regions_style = {
    'color': 'black',
    'width': 1,
    'fillColor': '00000000'
}

# === 2) POPULATION (FOND) ===
vis_pop = {
    'min': 0,
    'max': 1000,
    'palette': ['blue','green','yellow','orange','red']
}

# === 3) BUFFERS autour des points de santé ===
buf1km  = sante.map(lambda f: f.buffer(1000))
buf5km  = sante.map(lambda f: f.buffer(5000))
buf10km = sante.map(lambda f: f.buffer(10000))

# === 4) CARTE ===
m = geemap.Map()
m.add_basemap('OpenStreetMap')
m.center_object(tcd0, 5)

# --- FOND : Population 100 m ---
m.addLayer(pop100_tchad, vis_pop, "Population 100m")

# --- Choroplèthe nb_sante ---
m.addLayer(nb_sante_img.clip(tcd0), vis_nb, "Nb de services de santé / région")

# --- Limites régionales ---
m.addLayer(tcd1.style(**regions_style), {}, "Régions")

# --- Buffers ---
m.addLayer(buf10km, {'color': 'yellow'}, "Buffer 10 km")
m.addLayer(buf5km,  {'color': 'orange'}, "Buffer 5 km")
m.addLayer(buf1km,  {'color': 'red'},    "Buffer 1 km")

# --- Points de santé ---
m.addLayer(sante, {'color': 'blue', 'pointRadius': 4}, "Points santé")
#Recording
output_html = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\buffer_sante.html"
m.to_html(output_html)
m


nb_sante max : 108


Map(center=[15.306204623659646, 18.656507734823464], controls=(WidgetControl(options=['position', 'transparent…

In [20]:
sante.first().getInfo()


{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [15.077197204831219, 12.133628662824206]},
 'id': '000000000000000002fa',
 'properties': {'code': 2101,
  'fclass': 'pharmacy',
  'name': 'Dar Silla',
  'osm_id': '2613381554'}}

In [21]:
# --- RASTER POPULATION 100m déjà chargé ---
# pop100 = ee.Image("projects/project-2026-476714/assets/tcd_pop_2025_CN_100m_R2025A_v1")
pop100 = ee.Image("projects/project-2026-476714/assets/tcd_pop_2025_CN_100m_R2025A_v1")
# On clippe au Tchad
pop100_tchad = pop100.clip(tcd0)

# Récupérer le nom du band automatiquement
band_pop = pop100_tchad.bandNames().get(0)


# =====================================================
# 1) Fonction : calculer population desservie par buffer
# =====================================================
def pop_in_buffer(feature):
    geom = feature.geometry()

    # Buffers
    buf1  = geom.buffer(1000)
    buf5  = geom.buffer(5000)
    buf10 = geom.buffer(10000)

    # Somme population dans chaque buffer
    pop1 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf1,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    pop5 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf5,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    pop10 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf10,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    return feature.set({
        "pop_1km":  pop1,
        "pop_5km":  pop5,
        "pop_10km": pop10
    })


# =====================================================
# 2) Application à tous les postes de santé
# =====================================================

sante_pop = sante.map(pop_in_buffer)

# =====================================================
# 3) Récupération côté client -> DataFrame
# =====================================================

info = sante_pop.getInfo()
rows = [f["properties"] for f in info["features"]]

df = pd.DataFrame(rows)

output_path = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs"
os.makedirs(output_path, exist_ok=True)

# Chemins des fichiers
csv_path  = os.path.join(output_path, "pop_buffer_sante.csv")
xlsx_path = os.path.join(output_path, "pop_buffer_sante.xlsx")

# Sauvegarde
df.to_csv(csv_path, index=False)
df.to_excel(xlsx_path, index=False)

print("Fichiers enregistrés :")
print(csv_path)
print(xlsx_path)


# Afficher les colonnes utiles
df[['name','pop_1km', 'pop_5km', 'pop_10km']]


Fichiers enregistrés :
C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\pop_buffer_sante.csv
C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\pop_buffer_sante.xlsx


,name,pop_1km,pop_5km,pop_10km
0,Dar Silla,34346.426617,770171.646437,1.507759e+06
1,Dépôt Pharmaceutique,28880.104013,605101.168661,1.512560e+06
2,Pharmacie Eguni,33550.427701,436430.797584,1.488296e+06
3,Sopivet,32867.095873,602328.535125,1.494927e+06
4,Dépôt pharmaceutique BERDEI,27228.251859,408864.838244,1.463440e+06
...,...,...,...,...
335,,689.415883,1909.898538,7.078187e+03
336,,4334.567497,8343.275835,1.247797e+04
337,Clinique dentaire Adventiste BARAKA,38850.055562,728282.036182,1.417409e+06
338,Cabinet dentaire ONEK,35634.789818,623859.386096,1.397711e+06


In [22]:

# Filtrer les POI pour ne garder que les écoles / éducation
ecoles = pois_p.filter(
    ee.Filter.inList(
        'fclass',
        ['school', 'college', 'university', 'kindergarten']
    )
)

print("Nombre de points d'écoles :", ecoles.size().getInfo())


Nombre de points d'écoles : 313


In [23]:
regions_stats_ecoles = tcd1.map(
    lambda region: region.set(
        "nb_ecoles",
        ecoles.filterBounds(region.geometry()).size()
    )
)


In [24]:
# === 1) CHOROPLÈTHE nb_ecoles ===
nb_ecoles_img = regions_stats_ecoles.reduceToImage(
    properties=['nb_ecoles'],
    reducer=ee.Reducer.first()
)

max_ecoles = regions_stats_ecoles.aggregate_max('nb_ecoles').getInfo()
print("nb_ecoles max :", max_ecoles)

vis_ecoles = {
    'min': 0,
    'max': max_ecoles,
    'palette': ['#fff5f0','#fcbba1','#fc9272','#ef3b2c','#99000d']  # rouge clair → foncé
}

regions_style = {
    'color': 'black',
    'width': 1,
    'fillColor': '00000000'
}

# === 2) POPULATION (FOND) ===
vis_pop = {
    'min': 0,
    'max': 1000,
    'palette': ['blue','green','yellow','orange','red']
}

# === 3) BUFFERS autour des écoles ===
buf1km_ec  = ecoles.map(lambda f: f.buffer(1000))
buf5km_ec  = ecoles.map(lambda f: f.buffer(5000))
buf10km_ec = ecoles.map(lambda f: f.buffer(10000))

# === 4) CARTE ===
m = geemap.Map()
m.add_basemap('OpenStreetMap')
m.center_object(tcd0, 5)

# Fond : population 100 m
m.addLayer(pop100_tchad, vis_pop, "Population 100m")

# Choroplèthe : nb d'écoles par région
m.addLayer(nb_ecoles_img.clip(tcd0), vis_ecoles, "Nb d'écoles / région")

# Limites régionales
m.addLayer(tcd1.style(**regions_style), {}, "Régions")

# Buffers (10 km, 5 km, 1 km) autour des écoles
m.addLayer(buf10km_ec, {'color': 'yellow'}, "Buffer 10 km (écoles)")
m.addLayer(buf5km_ec,  {'color': 'orange'}, "Buffer 5 km (écoles)")
m.addLayer(buf1km_ec,  {'color': 'red'},    "Buffer 1 km (écoles)")

# Points d'écoles
m.addLayer(ecoles, {'color': 'blue', 'pointRadius': 4}, "Écoles (OSM)")
output_html = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\buffer_ecoles.html"
m.to_html(output_html)
m


nb_ecoles max : 126


Map(center=[15.306204623659646, 18.656507734823464], controls=(WidgetControl(options=['position', 'transparent…

In [25]:
# === 1) Fonction : population desservie autour d'une école ===
def pop_in_buffer_ecole(feature):
    geom = feature.geometry()

    buf1  = geom.buffer(1000)
    buf5  = geom.buffer(5000)
    buf10 = geom.buffer(10000)

    pop1 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf1,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    pop5 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf5,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    pop10 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf10,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    return feature.set({
        "pop_1km":  pop1,
        "pop_5km":  pop5,
        "pop_10km": pop10
    })


In [26]:
ecoles_pop = ecoles.map(pop_in_buffer_ecole)


In [27]:
info_ec = ecoles_pop.getInfo()
rows_ec = [f["properties"] for f in info_ec["features"]]

df_ecoles = pd.DataFrame(rows_ec)
output_path = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs"
os.makedirs(output_path, exist_ok=True)

# Chemins des fichiers
csv_path  = os.path.join(output_path, "pop_buffer_ecoles.csv")
xlsx_path = os.path.join(output_path, "pop_buffer_ecoles.xlsx")

# Sauvegarde
df.to_csv(csv_path, index=False)
df.to_excel(xlsx_path, index=False)

print("Fichiers enregistrés :")
print(csv_path)
print(xlsx_path)
df_ecoles[['name','pop_1km','pop_5km','pop_10km']]


Fichiers enregistrés :
C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\pop_buffer_ecoles.csv
C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\pop_buffer_ecoles.xlsx


,name,pop_1km,pop_5km,pop_10km
0,Institut ISCAM,30742.334093,640049.723867,1.506566e+06
1,Rectorat Universite de sarh,938.049349,18671.521554,1.354510e+05
2,قسم العلوم بجامعة الملك فيصل,36902.586381,638562.043326,1.530162e+06
3,FATEK,282.727120,17740.750217,1.182922e+05
4,Faculté des Sciences Exactes et Appliquées,14079.062353,277243.668132,9.384541e+05
...,...,...,...,...
308,Complexe Abou Hourayra,6139.361943,169546.058789,3.730090e+05
309,Académie Royale,21788.876166,475315.594616,1.409743e+06
310,ISFOP,23540.260324,484238.071413,1.416895e+06
311,Ecole des métiers d'administration et de commerce,28827.216018,582494.591629,1.383367e+06


In [28]:


# Lieux de culte (points OSM)
culte = pofw_p.merge(pofw_a)


print("Nombre de lieux de culte :", culte.size().getInfo())


Nombre de lieux de culte : 313


In [29]:
regions_stats_culte = tcd1.map(
    lambda region: region.set(
        "nb_culte",
        culte.filterBounds(region.geometry()).size()
    )
)


In [30]:
# === 1) CHOROPLÈTHE nb_culte ===
nb_culte_img = regions_stats_culte.reduceToImage(
    properties=['nb_culte'],
    reducer=ee.Reducer.first()
)

max_culte = regions_stats_culte.aggregate_max('nb_culte').getInfo()
print("nb_culte max :", max_culte)

vis_culte = {
    'min': 0,
    'max': max_culte,
    'palette': ['#f7fcf5','#c7e9c0','#74c476','#238b45','#00441b']  # vert clair → foncé
}

regions_style = {
    'color': 'black',
    'width': 1,
    'fillColor': '00000000'
}

# === 2) POPULATION (FOND) ===
vis_pop = {
    'min': 0,
    'max': 1000,
    'palette': ['blue','green','yellow','orange','red']
}

# === 3) BUFFERS autour des lieux de culte ===
buf1km_cu  = culte.map(lambda f: f.buffer(1000))
buf5km_cu  = culte.map(lambda f: f.buffer(5000))
buf10km_cu = culte.map(lambda f: f.buffer(10000))

# === 4) CARTE ===
m = geemap.Map()
m.add_basemap('OpenStreetMap')
m.center_object(tcd0, 5)

# Fond : population 100 m
m.addLayer(pop100_tchad, vis_pop, "Population 100m")

# Choroplèthe : nb de lieux de culte par région
m.addLayer(nb_culte_img.clip(tcd0), vis_culte, "Nb de lieux de culte / région")

# Limites régionales
m.addLayer(tcd1.style(**regions_style), {}, "Régions")

# Buffers (10 km, 5 km, 1 km) autour des lieux de culte
m.addLayer(buf10km_cu, {'color': 'yellow'}, "Buffer 10 km (culte)")
m.addLayer(buf5km_cu,  {'color': 'orange'}, "Buffer 5 km (culte)")
m.addLayer(buf1km_cu,  {'color': 'red'},    "Buffer 1 km (culte)")

# Points de lieux de culte
m.addLayer(culte, {'color': 'purple', 'pointRadius': 4}, "Lieux de culte (OSM)")
output_html = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\buffer_culte.html"
m.to_html(output_html)
m


nb_culte max : 77


Map(center=[15.306204623659646, 18.656507734823464], controls=(WidgetControl(options=['position', 'transparent…

In [31]:
# === Fonction : population desservie autour d'un lieu de culte ===
def pop_in_buffer_culte(feature):
    geom = feature.geometry()

    buf1  = geom.buffer(1000)
    buf5  = geom.buffer(5000)
    buf10 = geom.buffer(10000)

    pop1 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf1,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    pop5 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf5,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    pop10 = pop100_tchad.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=buf10,
        scale=100,
        maxPixels=1e13
    ).get(band_pop)

    return feature.set({
        "pop_1km":  pop1,
        "pop_5km":  pop5,
        "pop_10km": pop10
    })


In [32]:
culte_pop = culte.map(pop_in_buffer_culte)


In [33]:
info_cu = culte_pop.getInfo()
rows_cu = [f["properties"] for f in info_cu["features"]]

df_culte = pd.DataFrame(rows_cu)

output_path = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs"
os.makedirs(output_path, exist_ok=True)

# Chemins des fichiers
csv_path  = os.path.join(output_path, "pop_buffer_culte.csv")
xlsx_path = os.path.join(output_path, "pop_buffer_culte.xlsx")

# Sauvegarde
df.to_csv(csv_path, index=False)
df.to_excel(xlsx_path, index=False)

print("Fichiers enregistrés :")
print(csv_path)
print(xlsx_path)

# Afficher TOUTES les lignes (attention, ça peut être long)
df_culte[['name','pop_1km','pop_5km','pop_10km']]


Fichiers enregistrés :
C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\pop_buffer_culte.csv
C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs\pop_buffer_culte.xlsx


,name,pop_1km,pop_5km,pop_10km
0,Eglise Baptiste mid Mission's Centrale,32069.800637,661000.253289,1.452443e+06
1,,8514.537306,42476.205951,6.016393e+04
2,Entente des Eglises et Missions Evangeliques a...,17135.337422,451032.007017,1.390843e+06
3,Cathédrale de Mongo كاتدرائية مونقو القديس إغن...,17714.693891,105758.558715,1.207576e+05
4,Paroisse Sainte Famille,38718.858331,654455.257141,1.392507e+06
...,...,...,...,...
308,Mosquée centrale,3308.595425,13518.666804,1.548131e+04
309,Djamiya Numero الجامع نوميرو,4769.399316,13689.032295,1.575973e+04
310,المسجد المركزي برداي,2073.365165,2727.516025,3.142525e+03
311,مسجد وارا Masdjid Ouara,37030.635679,676057.440542,1.445748e+06


In [34]:
# Collection des aires protégées (asset GEE)
wdpa = ee.FeatureCollection("projects/project-2026-476714/assets/WDPA_TCD_polygons_merged")

# Régions administratives (niveau 1, GAUL simplifié)
regions = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") \
    .filter(ee.Filter.eq("ADM0_NAME", "Chad"))

# Géométrie du pays
tchad_geom = regions.geometry()


In [35]:
m = geemap.Map(center=[15, 19], zoom=5)

# Style des régions
style_regions = {
    "color": "1f4e79",       # bleu foncé
    "fillColor": "00000000", # transparent
    "width": 1.5
}

# Style des aires protégées (ROUGE)
style_wdpa = {
    "color": "b30000",        # rouge foncé (contour)
    "fillColor": "ff000050",  # rouge transparent
    "width": 1
}

# Ajout des calques
m.addLayer(regions.style(**style_regions), {}, "Régions")
m.addLayer(wdpa.style(**style_wdpa), {}, "Aires protégées (rouge)")
output_path = r"C:\Users\admin\Pictures\ISEP3\TP2\outputs"
os.makedirs(output_path, exist_ok=True)

html_path = os.path.join(output_path, "carte_aires_protegees_tchad.html")
m.save(html_path)

print("Carte HTML enregistrée ici :")
print(html_path)

m


Carte HTML enregistrée ici :
C:\Users\admin\Pictures\ISEP3\TP2\outputs\carte_aires_protegees_tchad.html


Map(center=[15, 19], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tran…

In [43]:
# Fonction : ajoute le nombre d'AP intersectant chaque région
def add_count(feat):
    geom = feat.geometry()
    count = wdpa.filterBounds(geom).size()
    return feat.set("count", count)

# Appliquer la fonction à chaque région
wdpa_by_region = regions.map(add_count)

# Conversion en DataFrame
df_regions = geemap.ee_to_df(wdpa_by_region)

# Colonnes utiles
df_regions = df_regions[["ADM1_NAME", "count"]].sort_values("count", ascending=False)
# --------------------------------------------------------
# 3) Enregistrement local (CSV + Excel)
# --------------------------------------------------------
output_path = r"C:\Users\admin\Pictures\ISEP3\TP2\outputs"
os.makedirs(output_path, exist_ok=True)

csv_path  = os.path.join(output_path, "wdpa_par_region.csv")
xlsx_path = os.path.join(output_path, "wdpa_par_region.xlsx")

df_regions.to_csv(csv_path, index=False)
df_regions.to_excel(xlsx_path, index=False)
df_regions


,ADM1_NAME,count
5,Baguirmi,2
27,Salamat,1
26,Logone Occidental,1
25,Lac,1
20,Sila,1
11,Mont De Lam,1
15,Barh Koh,0
24,Guera,0
23,Biltine,0
22,Tandjile Ouest,0


In [44]:
# Raster population 100 m (déjà utilisé plus haut)
pop100 = ee.Image("projects/project-2026-476714/assets/tcd_pop_2025_CN_100m_R2025A_v1")
pop100_tchad = pop100.clip(tcd0)

# Nom du band (b1 en général)
band_pop = pop100_tchad.bandNames().get(0)
print("Band population utilisé :", band_pop.getInfo())


Band population utilisé : b1


In [45]:
def add_pop_in_wdpa(feat):
    geom = feat.geometry()

    pop_total = pop100_tchad.reduceRegion(
        reducer   = ee.Reducer.sum(),
        geometry  = geom,
        scale     = 100,    # raster 100 m
        maxPixels = 1e13
    ).get(band_pop)

    # On ajoute un attribut "pop_in"
    return feat.set("pop_in", pop_total)


In [46]:
wdpa_pop = wdpa.map(add_pop_in_wdpa)


In [47]:
info_wdpa = wdpa_pop.getInfo()
rows_wdpa = [f["properties"] for f in info_wdpa["features"]]

df_wdpa = pd.DataFrame(rows_wdpa)

# Afficher quelques colonnes utiles (à adapter selon tes champs WDPA)
df_wdpa[['NAME', 'STATUS', 'IUCN_CAT', 'pop_in']].head()


,NAME,STATUS,IUCN_CAT,pop_in
0,Partie tchadienne du lac Tchad,Designated,Not Reported,0.000000
1,Plaines d'inondation du Logone et les dépressi...,Designated,Not Reported,0.000000
2,Plaines d'inondation des Bahr Aouk et Salamat,Designated,Not Reported,0.000000
3,Plaine de Massenya,Designated,Not Reported,0.000000
4,Beinamar,Proposed,Not Reported,59.286896


In [49]:
# Conversion de la feature collection wdpa_pop en DataFrame
info_wdpa = wdpa_pop.getInfo()
rows_wdpa = [f["properties"] for f in info_wdpa["features"]]

df_wdpa = pd.DataFrame(rows_wdpa)

# Dossier de sortie
output_path = r"C:\Users\admin\Pictures\ISEP3\SUITE DU TP2\outputs"
os.makedirs(output_path, exist_ok=True)

# Chemins des fichiers
csv_path  = os.path.join(output_path, "pop_protected_areas.csv")
xlsx_path = os.path.join(output_path, "pop_protected_areas.xlsx")

# 🔴 ICI on enregistre bien df_wdpa (et plus df_regions)
df_wdpa.to_csv(csv_path, index=False)
df_wdpa.to_excel(xlsx_path, index=False)

# Afficher quelques colonnes utiles (dans un notebook ça affiche le tableau)
df_wdpa[['NAME', 'STATUS', 'IUCN_CAT', 'pop_in']]


,NAME,STATUS,IUCN_CAT,pop_in
0,Partie tchadienne du lac Tchad,Designated,Not Reported,0.000000
1,Plaines d'inondation du Logone et les dépressi...,Designated,Not Reported,0.000000
2,Plaines d'inondation des Bahr Aouk et Salamat,Designated,Not Reported,0.000000
3,Plaine de Massenya,Designated,Not Reported,0.000000
4,Beinamar,Proposed,Not Reported,59.286896
5,Goz-Beida,Proposed,Not Reported,0.000000
6,Larmanaye,Proposed,Not Reported,0.000000
